In [48]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%ls


In [ ]:
%cd 

# **INFOL3 - SDD- UY1 - Departement d'informatique**

# **IMPLEMENTATION DU TUTORIEL SUR L'ALGORITHME KNN**

 >>>  [*Classificateur de courriers indesirables avec l'algorithme knn*](https://towardsdatasciencecomspam-email-classifier-with-knn-from-scratch-python-6e68eeb50a9e)




                                   
### Noms des membres du groupe (**Groupe 4**) :
  1. NOUCHEN TCHAMBA Parnell Voltaire - 19M2326
  2. DJEUMEZA DONGMO Julie Merveille - 19M2105
  3. MADJOU Alvine Patricia - 19M2103
  4. NAKAM YOPDUP Manuella Kristeva - 19M2233

KNN est un algorithme d'apprentissage supervisé, basé sur la classification des objets, dans le cardre de ce tutoriel, nous allons l'utiliserons pour classer les mails (spam ou pas).

1. **Importation** **des** **librairies** 

In [49]:
import os
import string
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np

Par ce bout de code nous importons toutes les bibliotheques qui seront necessaires pour l'implementation de du tutoriel.

2. **Chargement** **des** **donnees**

In [50]:
def load_data():
    print("Loading data...")
    
    ham_files_location = os.listdir("/content/drive/MyDrive/data_knn/enron2/ham/") # repertoire propres a cette machine
    spam_files_location = os.listdir("/content/drive/MyDrive/data_knn/enron2/spam/") # idem, veuillew modifier pour tester dans les votres
    data = []
    # Chargement des email ham
    for file_path in ham_files_location:
        print(file_path)
        f = open("/content/drive/MyDrive/data_knn/enron2/ham/" + file_path, "r", encoding='utf-8', errors='ignore')
        text = str(f.read())
        data.append([text, "ham"])

    # Chargement des emails spam 
    for file_path in spam_files_location:
        f = open("/content/drive/MyDrive/data_knn/enron2/spam/" + file_path, "r", encoding='utf-8', errors='ignore')
        text = str(f.read())
        data.append([text, "spam"])

    #On transforme les donnees en un tableau numpy
    data = np.array(data)
    print("flag 1: loaded data")
    return data




La fonction load_data ci dessus a pour role de recuperer tout les noms de fichier des fichiers texte contenus dans les dossiers ham et spam. 
Par la suite la liste de noms de fichiers texte contenus dans chacun d'eux sera parcouru. Chacun de leur fichier sera ensuite ouvert en mode lecture, le texte de l'email qui y est contenu sera ainsi lu sous forme de chaine de caratère  et stocké dans une variable(text). Une liste formé de ce texte et de son etiquette associée(ham/spam) formeront une liste qui sera ajouté dans data.

3. **Pretraitement** **des** **donnees**

In [51]:
# Suppression des bruits

def preprocess_data(data):
  print("Preprocessing data...")
  punc = string.punctuation           # liste des ponctuations
  sw = stopwords.words('english')     # liste des mots vides

  # Suppression des ponctuations et des symboles en les remplacant par une chaine vide
  for record in data:     
      for item in punc:
          record[0] = record[0].replace(item, "")

      # Transformations de toute les lettre en miniscule et suppression des mots vides 
      splittedWords = record[0].split() #liste de tout les mots de l'email
      newText = ""
      for word in splittedWords:
          if word not in sw:
              word = word.lower()
              newText = newText + " " + word # On ajoute dans la variable newText les mots n'etant pas des mots vides et qui ont ete transformes en miniscule      
      record[0] = newText # On reaffecte a record[0] les donnees qui ont été pretraitées 
      
  print("flag 2: preprocessed data")        
  return data



La fonction preprocess_data() ci-dessus effectue le prétraitement en supprimant les ponctuations et symboles , en transformant toutes les lettres en minuscule et en supprimant les mots vides pour chaque enregistrement dans data

4. **Separation des donnees en donnees d'apprentissage et en donnees de test**

In [52]:
def split_data(data):
    print("Splitting data...")
    
    features = data[:, 0]   # Tableau contenant tout les textes d'email
    labels = data[:, 1]     # Tableau contenant les etiquettes correspondantes
    print(labels)
    training_data, test_data, training_labels, test_labels =\
        train_test_split(features, labels, test_size = 0.27, random_state = 42)# On divise features et labels en donnees d'entrainement et de test où les donnees de test sont de 27%
    
    print("flag 3: splitted data")
    return training_data, test_data, training_labels, test_labels

La fonction split_data ( ) ci-dessus fractionne l'ensemble des données en un ensemble d'apprentissage constitué de trainng_data et training_labels et en un ensemble de test qui lui est constitué de test_data et test_labels

Ecrivons maintenant la fonction permettant de compter le nombre d'occurence d'un mot dans un mail

In [53]:
# pour compter le nombre de mots et leur occurences dans la chaine text, et retourner un dictionnaire contenant un mot et son occurence
def get_count(text):
    wordCounts = dict()
    for word in text.split():
        if word in wordCounts:
            wordCounts[word] += 1
        else:
            wordCounts[word] = 1
    
    return wordCounts

La fonction suivante est la fonction qui calcule la distance entre deux mails, 

In [54]:
"""
calcule la distance entre deux mail (notamment celui de test et un de train)
en utilisant la distance euclidienne, d'abord il fait la somme au carre de la difference des occurences du mots appartenant dans les deux mails, s'il n'est pas dans les deux a la fois, il fait juste le carre de l'occurence du mot, qu'il soit dans test ou train
ensuite il retourne la racine carre de la somme (de total en fait)
"""

def euclidean_difference(test_WordCounts, training_WordCounts):
  total = 0
  for word in test_WordCounts:
    if word in test_WordCounts and word in training_WordCounts:
      total += (test_WordCounts[word] - training_WordCounts[word])**2
      del training_WordCounts[word]
    else:
      total += test_WordCounts[word]**2
  for word in training_WordCounts:
    total += training_WordCounts[word]**2
  return total**0.5

Plus bas la fonction calcule la classe d'une entite à partir de l'ensemble des mails les plus proches (ses plus proches voisins), elle retourne soit "ham", si le mail est un ham ou "spam" si non.

In [55]:
# pour avoir a partir des kplus proches voisins, la classe de l'entite recherchee
def get_class(selected_Kvalues):
    spam_count = 0
    ham_count = 0
    for value in selected_Kvalues:
        if value[0] == "spam":
            spam_count += 1
        else:
            ham_count += 1
    if spam_count > ham_count:
      return "spam"
    else:
      return "ham"

Maintenant, ecrivons la fonctions de classificateur des k plus proches voisins: Elle prend en entree l'ensemble des mails d'entrainement, les etiquettes correspondantes, le mail a classer /l'ensembles des mails a classer, la valeur de k

In [56]:
def knn_classifier(training_data, training_labels, test_data, K, tsize):
  print("Running KNN classifier...")
  result = []
  counter = 1
  training_wordCounts = []

  #Pour chaque texte de l'email dans les donnees d'apprentissage, on recupere le dictionnaire correspondants a l'ensemble de ses mots et leur occurencesqu'on va ajouter a la liste contenant le doctionnaire mots:nombre d'eccurences pour tous les mails de l'ensemble de test 
  for training_text in training_data:
    training_WordCounts.append(get_count(training_text))

  #Ici on fait pareil pour l'ensemble des emails de test, on recupere le dictionnaire mot:nombre d'occurence
  for test_text in test_data:
    similarity = []
    test_WordCounts = get_count(test_text)

    #ensuite on va calculer la similarite entre ce mail et tous les mails de l'ensemble de test, et on ajoute la valeur de la similarite dans la liste "similarity" precede de l'email avec lequel la similarite a ete calculee
    for index in range(len(training_data)):
        euclidean_diff = euclidean_difference(test_WordCounts, training_WordCounts[index])
        similarity.append([training_labels[index], euclidean_diff])
    similarity = sorted(similarity, key = lambda i:i[1]) # ensuite on tri par ordre de plus simlaire le mail de test avec les mails d'entrainement

    # on selectionne les k voisins (sachant qu'en fait ce sont les k premiers mails, pris dans la liste similarity car elle a deja ete cree)
    selected_kvalues = []
    for i in rang (K):
      selected_Kvalues.append(similarity[i])
    result.append(get_class(selected_Kvalues)) #puis on calcule la classe de l''email de test correspondante
  return result # on retourne la liste des classe des emails de test



  

 Cette fonction classificateur permet de predire la classe d'un email a partir de la similarite entre un email de test et chaque email d'entrainement. 

**Programme principal**

Ici nous allons resumer toutes ces etapes decrites plus haut en appellant juste les fonctions dans l'ordre. Le but est de pouvoir calculer les metriques pour cet algorithme. 

la fonction accuracy_score compare directement la difference entre les mails bien predits et les mails mal predits. K est le nombre de voisins que l'on devra choisir

In [57]:
def main(K):
    data = load_data() # chargement des donnees
    data = preprocess_data(data) # traitement
    training_data, test_data, training_labels, test_labels = split_data(data) # division des donnees
    tsize = len(test_data)
    result = knn_classifier(training_data, training_labels, test_data[:tsize], K, tsize) 
    accuracy = accuracy_score(test_labels[:tsize], result)
    # affichons quelques elements de reponses
    print("training data size\t: " + str(len(training_data)))
    print("test data size\t\t: " + str(len(test_data)))
    print("K value\t\t\t\t: " + str(K))
    print("Samples tested\t\t: " + str(tsize))
    print("% accuracy\t\t\t: " + str(accuracy * 100))
    print("Number correct\t\t: " + str(int(accuracy * tsize)))
    print("Number wrong\t\t: " + str(int((1 - accuracy) * tsize)))

Appel de la fonction main pour les test 

In [ ]:
main(10)

Loading data...
1171.2000-05-02.kaminski.ham.txt
1187.2000-05-08.kaminski.ham.txt
1215.2000-05-11.kaminski.ham.txt
1128.2000-04-27.kaminski.ham.txt
1124.2000-04-27.kaminski.ham.txt
1117.2000-04-26.kaminski.ham.txt
1136.2000-04-28.kaminski.ham.txt
1114.2000-04-26.kaminski.ham.txt
1113.2000-04-26.kaminski.ham.txt
1115.2000-04-26.kaminski.ham.txt
1123.2000-04-27.kaminski.ham.txt
1138.2000-04-28.kaminski.ham.txt
1122.2000-04-27.kaminski.ham.txt
1121.2000-04-27.kaminski.ham.txt
1126.2000-04-27.kaminski.ham.txt
1125.2000-04-27.kaminski.ham.txt
1120.2000-04-27.kaminski.ham.txt
1098.2000-04-25.kaminski.ham.txt
1085.2000-04-24.kaminski.ham.txt
1100.2000-04-25.kaminski.ham.txt
1099.2000-04-25.kaminski.ham.txt
1052.2000-04-19.kaminski.ham.txt
1083.2000-04-24.kaminski.ham.txt
1084.2000-04-24.kaminski.ham.txt
1051.2000-04-19.kaminski.ham.txt
1101.2000-04-25.kaminski.ham.txt
1082.2000-04-24.kaminski.ham.txt
1054.2000-04-19.kaminski.ham.txt
1007.2000-04-14.kaminski.ham.txt
0945.2000-04-10.kaminski.ha